# AutoBET: Predictive Soccer Outcome Modelling and Algorithmic Betting System

## Load data

In [85]:
import pandas as pd
import numpy as np
import sqlite3
import math

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

database_path = "../data/database.sqlite"

with sqlite3.connect(database_path) as con:
    sql = '''
    SELECT *
    FROM match 
    '''
    df_match = pd.read_sql_query(sql, con)
    
with sqlite3.connect(database_path) as con:
    sql = '''
    SELECT player_api_id, overall_rating
    FROM player_attributes
    '''
    player_lvl = pd.read_sql_query(sql, con)

# with sqlite3.connect(database_path) as con:
#     sql = '''
#     SELECT *
#     FROM team_attributes 
#     '''
#     df_team = pd.read_sql_query(sql, con)
    
# with sqlite3.connect(database_path) as con:
#     sql = '''
#     SELECT team_api_id, team_short_name
#     FROM team
#     '''
#     team_id = pd.read_sql_query(sql, con)
    

    
# with sqlite3.connect('../data/features.sqlite') as con:
#     sql = '''
#     SELECT *
#     FROM team_features
#     '''
#     team_lvl = pd.read_sql_query(sql, con)

## Preprocess data

Extract relevant data and join into single table

In [86]:
player_lvl = player_lvl.groupby('player_api_id').mean().to_dict()
def get_rating(player_id):
    if math.isnan(player_id):
        return -1
    return player_lvl['overall_rating'][player_id]

def get_outcome(home, away):
    if home > away:
        return 0
    elif home == away:
        return 1
    else:
        return 2

df = df_match[['stage', 'home_team_goal', 'away_team_goal','B365H','B365D','B365A','BWH',
               'BWD','BWA','IWH','IWD','IWA','LBH','LBD','LBA','WHH','WHD','WHA','SJH','SJD','SJA',
               'VCH','VCD','VCA','GBH','GBD','GBA']]

df_tmp = df_match[['home_player_1','home_player_2','home_player_3','home_player_4','home_player_5',
                   'home_player_6','home_player_7','home_player_8','home_player_9','home_player_10',
                   'home_player_11','away_player_1','away_player_2','away_player_3','away_player_4',
                   'away_player_5','away_player_6','away_player_7','away_player_8','away_player_9',
                   'away_player_10','away_player_11']]

df = pd.concat([df, df_tmp.applymap(lambda x: get_rating(x))], axis=1)
df['target'] = df.apply(lambda x: get_outcome(x['home_team_goal'], x['away_team_goal']), axis=1)
df = df.drop(['home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal'], axis=1)

df.dropna(inplace=True)
df.head()

,country_id,league_id,stage,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,target
0,1,1,1,1.73,3.40,5.00,1.75,3.35,4.20,1.85,3.2,3.5,1.80,3.3,3.75,1.70,3.30,4.33,1.90,3.3,4.00,1.65,3.40,4.50,1.78,3.25,4.00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
1,1,1,1,1.95,3.20,3.60,1.80,3.30,3.95,1.90,3.2,3.5,1.90,3.2,3.50,1.83,3.30,3.60,1.95,3.3,3.80,2.00,3.25,3.25,1.85,3.25,3.75,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
2,1,1,1,2.38,3.30,2.75,2.40,3.30,2.55,2.60,3.1,2.3,2.50,3.2,2.50,2.50,3.25,2.40,2.63,3.3,2.50,2.35,3.25,2.65,2.50,3.20,2.50,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2
3,1,1,1,1.44,3.75,7.50,1.40,4.00,6.80,1.40,3.9,6.0,1.44,3.6,6.50,1.44,3.75,6.00,1.44,4.0,7.50,1.45,3.75,6.50,1.50,3.75,5.50,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
4,1,1,1,5.00,3.50,1.65,5.00,3.50,1.60,4.00,3.3,1.7,4.00,3.4,1.72,4.20,3.40,1.70,4.50,3.5,1.73,4.50,3.40,1.65,4.50,3.50,1.65,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2


## Outcome prediction

In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesClassifier, AdaBoostClassifier
from operator import itemgetter
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold

%matplotlib inline

In [88]:
# create data for model
X = df.drop('target', axis=1).values
y = df['target'].values

# keep 15% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=33)

In [89]:
kf = KFold(n_splits=5)
for nfold, (train_index, test_index) in enumerate(kf.split(X_train)):
    print('Fold', nfold+1)
    
    # create train / validation split
    Xtr, Xte = X_train[train_index], X_train[test_index]
    ytr, yte = y_train[train_index], y_train[test_index]
    
    # define predictive model
    model = RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=4, max_features=40,
                                   max_leaf_nodes=None, min_impurity_split=1e-07, min_samples_leaf=3, 
                                   min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=500, 
                                   n_jobs=-1, oob_score=False, random_state=33, verbose=0, warm_start=False)
    
    # fit the model on the trainsplit
    model.fit(Xtr, ytr)
    
    # evaluate fold performance
    print(classification_report(yte, model.predict(Xte)))
    
    # predict testset
    y_pred = np.array(model.predict_proba(X_test))
    
    # sum fold predictions
    if nfold == 0:
        pred_df = y_pred
    else:
        pred_df += y_pred

# average fold predictions, create df
pred_df /= 5
pred_df = 1 / pd.DataFrame(pred_df, columns=['H', 'D', 'A'])

('Fold', 1)
             precision    recall  f1-score   support

          0       0.54      0.85      0.66      1098
          1       0.00      0.00      0.00       635
          2       0.49      0.49      0.49       656

avg / total       0.38      0.53      0.44      2389

('Fold', 2)
             precision    recall  f1-score   support

          0       0.55      0.84      0.67      1114
          1       0.00      0.00      0.00       610
          2       0.47      0.49      0.48       665

avg / total       0.39      0.53      0.44      2389

('Fold', 3)


C:\Users\Daniel\AppData\Local\Continuum\Miniconda2\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.56      0.82      0.66      1128
          1       0.00      0.00      0.00       601
          2       0.48      0.54      0.51       660

avg / total       0.40      0.54      0.45      2389

('Fold', 4)
             precision    recall  f1-score   support

          0       0.53      0.84      0.65      1063
          1       0.00      0.00      0.00       642
          2       0.49      0.50      0.49       684

avg / total       0.38      0.52      0.43      2389

('Fold', 5)
             precision    recall  f1-score   support

          0       0.55      0.83      0.66      1127
          1       0.00      0.00      0.00       568
          2       0.47      0.47      0.47       694

avg / total       0.40      0.53      0.45      2389



In [93]:
tmp = df.drop(['target'], axis=1)
pd.DataFrame(pd.concat([pd.Series(model.feature_importances_), pd.Series(tmp.columns)], axis=1)).sort_values(0, ascending=False)

,0,1
26,0.466079,GBA
15,0.173218,WHH
21,0.128384,VCH
18,0.097256,SJH
11,0.018075,IWA
6,0.014018,BWH
9,0.014014,IWH
37,0.008722,home_player_11
22,0.008358,VCD
36,0.007652,home_player_10


In [101]:
from sklearn import metrics

metrics.accuracy_score(y_pred=[np.argmin(x) for x in pred_df.values], y_true=y_test)

0.54717875770507352

### Automatic strategies

In [44]:
def strategy_randomforest1(row, binary=True):
    if binary: 
        if row["prob_h"] < row["B365H"]:
            return ["H", 1]
        elif row["prob_nh"] < 1/((1/row["B365D"]) + (1/row["B365A"])):
            return ["NH", 1]
        else:
            return ["H", 0]
    else:
        if row["prob_h"] < row["B365H"]:
            return ["H", 1]
        elif row["prob_d"] < row["B365D"]:
            return ["D", 1]
        elif row["prob_a"] < row["B365A"]:
            return ["A", 1]
        else:
            return ["H", 0]

def strategy_randomforest2(row, binary=True):
    if binary: 
        if row["prob_h"] < row["B365H"]:
            return ["H", 1]
        elif row["prob_nh"] < 1/((1/row["B365D"]) + (1/row["B365A"])):
            return ["NH", 1]
        else:
            return ["H", 0]

def strategy_randomforest1_hwins(row, binary=True):
    if row["prob_h"] < row["B365H"]:
        return ["H", 1]
    else:
        return ["H", 0]
    
def strategy_dynamic_betting(row, binary=True):
    if binary:
        oddsNH = 1/((1/row["B365D"]) + (1/row["B365A"]))
        if row['prob_h'] < row['B365H']:
            return ['H', 2**(row['B365H'] / row['prob_h'])]
        elif row["prob_nh"] < oddsNH:
            return ["NH", 2**(oddsNH / row['prob_h'])]
        else:
            return ["H", 0]

In [60]:
xtest = pd.DataFrame(X_test, columns=[colname for colname in df.columns if colname not in ['index', 'target']])

In [62]:
odds = xtest.iloc[:, :24]

odds_A = odds[[colname for colname in odds.columns if colname[-1] == 'A']]
odds_H = odds[[colname for colname in odds.columns if colname[-1] == 'H']]
odds_D = odds[[colname for colname in odds.columns if colname[-1] == 'D']]

In [84]:
from tqdm import *

# create dataframe for odd ratios
ratio_df = pd.DataFrame(index=odds.columns)
for row in tqdm(range(len(odds_A))):
    
    # divide betting house odds and our odds for every class
    away = pd.DataFrame(odds_A.iloc[row, :] / pred_df.loc[row, 'A'])
    home = pd.DataFrame(odds_H.iloc[row, :] / pred_df.loc[row, 'H'])
    draw = pd.DataFrame(odds_D.iloc[row, :] / pred_df.loc[row, 'D'])
    
    # append ratios to new dataframe
    r = pd.concat([home, draw, away], axis=0)
    ratio_df = pd.concat([ratio_df, r], axis=1)
    


100%|██████████| 2077/2077 [00:10<00:00, 201.48it/s]


In [87]:
ratio_df = ratio_df.reset_index()

In [92]:
ratio_df[ratio_df.iloc[:, 1:].idxmax()]

,15,18,1,11,20,15,11,2,2,1,20,8,3,20,15,17,19,16,17,11,15,14,15,17,5,8,18,5,11,10,7,0,18,11,19,3,3,0,23,0,18,2,1,4,18,5,17,2,16,11,16,5,0,20,18,21,17,1,20,17,15,17,23,15,2,12,16,20,16,4,18,0,20,0,2,15,0,0,9,3,20,23,12,2,16,19,2,18,18,4,2,2,1,2,2,5,1,19,0,18,20,18,14,1,6,16,0,2,14,23,17,17,18,2,11,18,0,18,21,21,19,11,11,19,19,18,11,0,16,16,2,18,2,10,18,3,19,19,6,18,11,23,15,11,11,17,1,18,12,18,19,15,18,0,1,21,12,2,2,20,2,17,2,20,0,9,15,2,1,16,0,2,17,17,20,1,3,18,20,18,16,1,12,19,21,1,1,21,20,21,17,8,11,17,8,18,18,20,11,18,16,0,1,14,5,1,19,19,4,1,5,20,15,12,12,18,19,18,15,16,20,16,9,15,8,17,11,0,11,2,19,3,11,19,11,2,19,17,15,1,1,1,18,0,2,0,15,14,4,17,...,2,2,1,11,18,12,0,0,20,19,0,16,0,0,1,11,2,18,1,2,15,0,0,2,0,17,17,18,11,16,5,15,4,0,17,10,0,17,19,12,8,0,12,2,15,15,9,2,2,19,20,2,17,0,18,17,1,5,2,2,5,18,0,2,0,2,4,1,20,11,2,17,23,2,2,1,15,15,2,16,20,5,20,15,3,19,17,18,0,2,0,18,9,11,16,2,2,22,9,0,11,18,11,0,11,2,0,15,0,14,2,9,1,1,5,15,15,5,0,15,16,23,15,2,1,6,17,9,2,0,11,9,13,5,18,0,1,20,11,11,19,1,8,20,11,1,0,15,15,16,9,1,9,14,19,8,20,0,15,11,18,1,5,15,1,0,23,21,20,2,14,19,0,23,17,2,9,18,1,3,20,16,11,2,4,2,5,17,16,16,0,17,5,15,4,19,0,16,0,19,6,20,20,2,0,2,22,1,2,1,0,18,2,0,18,13,23,2,20,14,11,9,18,2,9,15,11,0,21,15,12,0,4,20,18,12,19,19,4,13,5,2,15,5,12,15,19,5,5,20
0,0.889529,0.901749,1.025411,0.838112,1.010527,0.889529,0.838112,1.023175,1.023175,1.025411,1.010527,0.743541,0.781772,1.010527,0.889529,0.983066,0.626110,0.933036,0.983066,0.838112,0.889529,0.929835,0.889529,0.983066,1.157404,0.743541,0.901749,1.157404,0.838112,0.893751,0.821540,0.969130,0.901749,0.838112,0.626110,0.781772,0.781772,0.969130,0.737148,0.969130,0.901749,1.023175,1.025411,0.996882,0.901749,1.157404,0.983066,1.023175,0.933036,0.838112,0.933036,1.157404,0.969130,1.010527,0.901749,1.019674,0.983066,1.025411,1.010527,0.983066,0.889529,0.983066,0.737148,0.889529,1.023175,0.956296,0.933036,1.010527,0.933036,0.996882,0.901749,0.969130,1.010527,0.969130,1.023175,0.889529,0.969130,0.969130,1.006499,0.781772,1.010527,0.737148,0.956296,1.023175,0.933036,0.626110,1.023175,0.901749,0.901749,0.996882,1.023175,1.023175,1.025411,1.023175,1.023175,1.157404,1.025411,0.626110,0.969130,0.901749,1.010527,0.901749,0.929835,1.025411,0.752615,0.933036,0.969130,1.023175,0.929835,0.737148,0.983066,0.983066,0.901749,1.023175,0.838112,0.901749,0.969130,0.901749,1.019674,1.019674,0.626110,0.838112,0.838112,0.626110,0.626110,0.901749,0.838112,0.969130,0.933036,0.933036,1.023175,0.901749,1.023175,0.893751,0.901749,0.781772,0.626110,0.626110,0.752615,0.901749,0.838112,0.737148,0.889529,0.838112,0.838112,0.983066,1.025411,0.901749,0.956296,0.901749,0.626110,0.889529,0.901749,0.969130,1.025411,1.019674,0.956296,1.023175,1.023175,1.010527,1.023175,0.983066,1.023175,1.010527,0.969130,1.006499,0.889529,1.023175,1.025411,0.933036,0.969130,1.023175,0.983066,0.983066,1.010527,1.025411,0.781772,0.901749,1.010527,0.901749,0.933036,1.025411,0.956296,0.626110,1.019674,1.025411,1.025411,1.019674,1.010527,1.019674,0.983066,0.743541,0.838112,0.983066,0.743541,0.901749,0.901749,1.010527,0.838112,0.901749,0.933036,0.969130,1.025411,0.929835,1.157404,1.025411,0.626110,0.626110,0.996882,1.025411,1.157404,1.010527,0.889529,0.956296,0.956296,0.901749,0.626110,0.901749,0.889529,0.933036,1.010527,0.933036,1.006499,0.889529,0.743541,0.983066,0.838112,0.969130,0.838112,1.023175,0.626110,0.781772,0.838112,0.626110,0.838112,1.023175,0.626110,0.983066,0.889529,1.025411,1.025411,1.025411,0.901749,0.969130,1.023175,0.969130,0.889529,0.929835,0.996882,0.983066,...,1.023175,1.023175,1.025411,0.838112,0.901749,0.956296,0.969130,0.969130,1.010527,0.626110,0.969130,0.933036,0.969130,0.969130,1.025411,0.838112,1.023175,0.901749,1.025411,1.023175,0.889529,0.969130,0.969130,1.023175,0.969130,0.983066,0.983066,0.901749,0.838112,0.933036,1.157404,0.889529,0.996882,0.969130,0.983066,0.893751,0.969130,0.983066,0.626110,0.956296,0.743541,0.969130,0.956296,1.023175,0.889529,0.889529,1.006499,1.023175,1.023175,0.626110,1.010527,